# Install Library yang diperlukan

Ini adalah script untuk mengubah pdf lampiran hasil CPNS jadi csv
Caveat: kalo langsung prosess 20rb halaman, agak keselek di tengah, jadi mending di-iterate beberapa halaman
Saya yakin ada cara yang lebih elegan. Feel free to give me any suggestions

Cara run </br>
python export_data_to_csv.py index_halaman_start index_halaman_end

Misal </br>
python export_data_to_csv.py 0 100

In [54]:
# import library
import tabula
import camelot
import pandas as pd
import datetime as dt
import sys

pd.options.display.max_columns = None

## Scraping Data

In [42]:
## Tenaga Puskesmas

data = {
    'Jenis Tenaga': ['Dokter atau dokter layanan primer', 'Dokter Gigi', 'Perawat', 'Bidan', 'Tenaga Kefarmasian', 'Tenaga Kesmas',
                     'Tenaga Kesehatan Lingkungan', 'Tenaga Gizi', 'Ahli Teknologi Laboratorium Medik'],
    'Jumlah Tenaga': [2097, 829, 9850, 13750, 889, 851, 674, 550, 514]
}

df = pd.DataFrame(data)

df['tahun'] = "2018"
df['unit_kerja'] = "puskesmas"
df

,Jenis Tenaga,Jumlah Tenaga,tahun,unit_kerja
0,Dokter atau dokter layanan primer,2097,2018,puskesmas
1,Dokter Gigi,829,2018,puskesmas
2,Perawat,9850,2018,puskesmas
3,Bidan,13750,2018,puskesmas
4,Tenaga Kefarmasian,889,2018,puskesmas
5,Tenaga Kesmas,851,2018,puskesmas
6,Tenaga Kesehatan Lingkungan,674,2018,puskesmas
7,Tenaga Gizi,550,2018,puskesmas
8,Ahli Teknologi Laboratorium Medik,514,2018,puskesmas


In [43]:
### Tenaga Rumah Sakit
    data = {
        'Jenis Tenaga': ['Dokter atau dokter layanan primer', 'Dokter Gigi', 'Perawat', 'Bidan', 'Tenaga Kefarmasian', 'Tenaga Kesmas',
                         'Tenaga Kesehatan Lingkungan', 'Tenaga Gizi', 'Ahli Teknologi Laboratorium Medik'],
        'Jumlah Tenaga': [12002, 2436, 31024, 5831, 5229, 572, 225, 778, 4947]
    }

    df = pd.DataFrame(data)

    df['tahun'] = "2018"
    df['unit_kerja'] = "Rumah Sakit"
df

,Jenis Tenaga,Jumlah Tenaga,tahun,unit_kerja
0,Dokter atau dokter layanan primer,12002,2018,Rumah Sakit
1,Dokter Gigi,2436,2018,Rumah Sakit
2,Perawat,31024,2018,Rumah Sakit
3,Bidan,5831,2018,Rumah Sakit
4,Tenaga Kefarmasian,5229,2018,Rumah Sakit
5,Tenaga Kesmas,572,2018,Rumah Sakit
6,Tenaga Kesehatan Lingkungan,225,2018,Rumah Sakit
7,Tenaga Gizi,778,2018,Rumah Sakit
8,Ahli Teknologi Laboratorium Medik,4947,2018,Rumah Sakit


In [45]:
## Rekapitulasi Rasio Tenaga Kesehatan /100.000

data = {
    'Tenaga Kesehatan': ['Dokter Spesialis', 'Dokter Umum', 'Dokter Gigi', 'Bidan', 'Perawat', 'Kefarmasian',
                         'Kesehatan Masyarakat', 'Kesehatan Lingkungan', 'Gizi', 'Keterafian Fisik',
                         'Keteknisan Medis', 'Non Kesehatan'],
    'Ratio/100.000 Penduduk': [17.36, 12.12, 7.1, 40.22, 81.1, 13.41, 3.17, 1.98, 2.7, 1.68, 6.6, 30.2 ]
}

df = pd.DataFrame(data)

df['tahun'] = "2018"
df

,Tenaga Kesehatan,Ratio/100.000 Penduduk,tahun
0,Dokter Spesialis,17.36,2018
1,Dokter Umum,12.12,2018
2,Dokter Gigi,7.10,2018
3,Bidan,40.22,2018
4,Perawat,81.10,2018
5,Kefarmasian,13.41,2018
6,Kesehatan Masyarakat,3.17,2018
7,Kesehatan Lingkungan,1.98,2018
8,Gizi,2.70,2018
9,Keterafian Fisik,1.68,2018


In [2]:
## Jumlah Fasilitas Kesehatan Menurut Kepemilikan 
### Data ini dicek secara manual pada setiap laporan kesehatan per kab/kota 

## Tidak tersedia

In [59]:
### PELAYANAN KESEHATAN GIGI DAN MULUT MENURUT KECAMATAN DAN PUSKESMAS

tables = camelot.read_pdf("2018.pdf", flavor='stream', pages='178')

df = tables[0].df

df = df.iloc[5:,1:]
df.reset_index(drop=True, inplace=True)
df.loc[len(df)] = ["Jawa Barat", "226974", "161703", "0.71"]
df.set_axis(["Kabupaten/Kota", "Jumlah Tumpatan Gigi Tetap", "Pencabutan Gigi Tetap", "Rasio Pencabutan"], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')
df = df.replace('#DIV/0!', '0')
for col in ['Rasio Pencabutan']:
    df[col] = df[col].str.replace(',', '.').astype(float)

for col in ['Jumlah Tumpatan Gigi Tetap', 'Pencabutan Gigi Tetap']:
    df[col] = df[col].str.replace('.', '').astype(int)
df['tahun'] = "2018"
df

<ipython-input-59-18e450a9c77d>:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '').astype(int)


,Kabupaten/Kota,Jumlah Tumpatan Gigi Tetap,Pencabutan Gigi Tetap,Rasio Pencabutan,tahun
0,Kabupaten Bogor,16998,13442,0.80,2018
1,Kabupaten Sukabumi,1410,2869,2.00,2018
2,Kabupaten Cianjur,5665,4422,0.80,2018
3,Kabupaten Bandung,23251,16984,0.70,2018
4,Kabupaten Garut,596,2421,4.10,2018
5,Kabupaten Tasikmalaya,1542,4256,2.80,2018
6,Kabupaten Ciamis,599,402,0.70,2018
7,Kabupaten Kuningan,2320,3673,1.60,2018
8,Kabupaten Cirebon,9859,10596,1.10,2018
9,Kabupaten Majalengka,2021,5089,2.50,2018


In [46]:
## PELAYANAN KESEHATAN GIGI DAN MULUT PADA ANAK SD DAN SETINGKAT MENURUT JENIS KELAMIN, KECAMATAN, DAN PUSKESMAS

tables = camelot.read_pdf("2018.pdf", flavor='stream', pages='179')

df = tables[0].df

df = df.iloc[7:,1:]
df.reset_index(drop=True, inplace=True)
df.iloc[27,0] = "Jawa Barat"
df.iloc[9,15] = "89.9"
df.set_axis(['Kabupaten/Kota', 'jumlah_SD/MI','jumlah_SD/MI_SG_Massal','%jumlah_SD/MI_SG_Massal',
             'jumlah_SD/MI_LG','%jumlah_SD/MI_LG',
            'jumlah_murid_SD/MI_L', 'jumlah_murid_SD/MI_P', 'jumlah_murid_SD/MI_LP',
            'jumlah_murid_SD/MI_L_Diperiksa','%jumlah_murid_SD/MI_L_Diperiksa','jumlah_murid_SD/MI_P_Diperiksa','%jumlah_murid_SD/MI_P_Diperiksa', 'jumlah_murid_SD/MI_LP_Diperiksa','%jumlah_murid_SD/MI_LP_Diperiksa',
            'jumlah_murid_SD/MI_L_PR','jumlah_murid_SD/MI_P_PR','jumlah_murid_SD/MI_LP_PR',
            'jumlah_murid_SD/MI_L_MP','%jumlah_murid_SD/MI_L_MP','jumlah_murid_SD/MI_P_MP','%jumlah_murid_SD/MI_P_MP','jumlah_murid_SD/MI_LP_MP','%jumlah_murid_SD/MI_LP_MP'], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')

## Replace #DIV N #Value
for col in df.columns:
    df.loc[df[col].astype(str).str.contains('#'), col] = 0

for col in df.columns[df.columns.str.contains('%')]:
    df[col] = df[col].str.replace(',', '.').astype(float)

for col in df.columns[~(df.columns.str.contains('%') | (df.columns == 'Kabupaten/Kota'))]:
    df[col] = df[col].replace('\.', '', regex=True).astype(int)
df.fillna(0, inplace=True)

tahun = "2018"
df['tahun'] = tahun
df.to_csv(f"dataset/pelayanan_gigi_anaksd/{tahun}.csv", index=False)
df

,Kabupaten/Kota,jumlah_SD/MI,jumlah_SD/MI_SG_Massal,%jumlah_SD/MI_SG_Massal,jumlah_SD/MI_LG,%jumlah_SD/MI_LG,jumlah_murid_SD/MI_L,jumlah_murid_SD/MI_P,jumlah_murid_SD/MI_LP,jumlah_murid_SD/MI_L_Diperiksa,%jumlah_murid_SD/MI_L_Diperiksa,jumlah_murid_SD/MI_P_Diperiksa,%jumlah_murid_SD/MI_P_Diperiksa,jumlah_murid_SD/MI_LP_Diperiksa,%jumlah_murid_SD/MI_LP_Diperiksa,jumlah_murid_SD/MI_L_PR,jumlah_murid_SD/MI_P_PR,jumlah_murid_SD/MI_LP_PR,jumlah_murid_SD/MI_L_MP,%jumlah_murid_SD/MI_L_MP,jumlah_murid_SD/MI_P_MP,%jumlah_murid_SD/MI_P_MP,jumlah_murid_SD/MI_LP_MP,%jumlah_murid_SD/MI_LP_MP,tahun
0,Kabupaten Bogor,2385,291,12.20,637,26.71,47762,46486,94248,8237,17.25,8478,18.24,16715,17.74,860,706,1566,97,11.28,76,10.76,173,11.05,2018
1,Kabupaten Sukabumi,1548,111,7.17,1433,92.57,27366,24820,52186,27014,98.71,24504,98.73,51518,98.72,0,0,0,0,0.00,0,0.00,0,0.00,2018
2,Kabupaten Cianjur,1197,523,43.69,713,59.57,80328,73062,153390,27307,33.99,30356,41.55,57663,37.59,11142,11041,22183,7714,69.23,8108,73.44,15822,71.32,2018
3,Kabupaten Bandung,1546,1546,100.00,758,49.03,0,0,0,0,0.00,0,0.00,0,0.00,0,0,0,0,0.00,0,0.00,0,0.00,2018
4,Kabupaten Garut,3805,780,20.50,532,13.98,2287,1518,3805,2287,100.00,1518,100.00,3805,100.00,253,279,532,253,100.00,279,100.00,532,100.00,2018
5,Kabupaten Tasikmalaya,5836,1242,21.28,5836,100.00,1769,1975,3744,787,44.49,1036,52.46,1823,48.69,4994,7498,12492,2029,40.63,3044,40.60,5073,40.61,2018
6,Kabupaten Ciamis,920,920,100.00,920,100.00,20027,18741,38768,9679,48.33,8945,47.73,18624,48.04,4947,3370,8317,2828,57.17,2536,75.25,5364,64.49,2018
7,Kabupaten Kuningan,738,205,27.78,229,31.03,33706,48584,82290,9332,27.69,9065,18.66,18397,22.36,2772,2604,5376,1355,48.88,1255,48.20,2610,48.55,2018
8,Kabupaten Cirebon,889,725,81.55,888,99.89,0,0,113500,0,0.00,0,0.00,100220,88.30,0,0,32940,0,0.00,0,0.00,13131,39.86,2018
9,Kabupaten Majalengka,759,712,93.81,712,93.81,34273,32996,67269,25830,75.37,25557,77.45,51387,76.39,899,8133,16213,4279,52.96,5137,63.16,9416,58.08,2018


In [47]:
## Jumlah Tenaga Medis Per Kab/Kota Puskesmas
tables = camelot.read_pdf("2018.pdf", flavor='stream', pages='231')

df = tables[0].df

df = df.iloc[:31, [1] + list(range(11, 17))]

df = df.iloc[4:,:]
df.set_axis(['Kab/Kota','jml_dg_l','jml_dg_p','jml_dg_lp','jml_dgsp_l','jml_dgsp_p','jml_dgsp_lp'], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')

for col in df.columns[~(df.columns.str.contains('%') | (df.columns == 'Kab/Kota'))]:
    df[col] = df[col].str.replace(',', '').astype(int)
df

,Kab/Kota,jml_dg_l,jml_dg_p,jml_dg_lp,jml_dgsp_l,jml_dgsp_p,jml_dgsp_lp
4,Kabupaten Bogor,5,65,70,0,0,0
5,Kabupaten Sukabumi,5,15,20,0,0,0
6,Kabupaten Cianjur,1,22,23,0,0,0
7,Kabupaten Bandung,12,56,68,0,0,0
8,Kabupaten Garut,6,15,21,0,0,0
9,Kabupaten Tasikmalaya,5,13,18,0,0,0
10,Kabupaten Ciamis,3,5,8,0,0,0
11,Kabupaten Kuningan,4,8,12,0,0,0
12,Kabupaten Cirebon,11,25,36,0,0,0
13,Kabupaten Majalengka,2,12,14,0,0,0


In [48]:
# Menggabungkan kolom 'jml_dg_l', 'jml_dg_p', 'jml_dgsp_l', dan 'jml_dgsp_p' menjadi satu kolom 'jumlah'
df_dg = pd.concat([
    df.loc[:,['Kab/Kota','jml_dg_l']].rename(columns={'jml_dg_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi', unit_kerja='puskesmas'),
    df.loc[:,['Kab/Kota','jml_dg_p']].rename(columns={'jml_dg_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi', unit_kerja='puskesmas'),
    df.loc[:,['Kab/Kota','jml_dgsp_l']].rename(columns={'jml_dgsp_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi spesialis', unit_kerja='puskesmas'),
    df.loc[:,['Kab/Kota','jml_dgsp_p']].rename(columns={'jml_dgsp_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi spesialis', unit_kerja='puskesmas')
], ignore_index=True)

# Menghapus duplikat dan mengurutkan data berdasarkan 'Kab/Kota' dan 'jns_kelamin'
df_dg_pks = df_dg.drop_duplicates().sort_values(['Kab/Kota','jns_kelamin'])

In [49]:
## Jumlah Tenaga Medis Kab/Kota Rumah Sakit

tables = camelot.read_pdf("2018.pdf", flavor='stream', pages='231')

df = tables[0].df

df = df.iloc[33:, [1] + list(range(11, 17))]

df.set_axis(['Kab/Kota','jml_dg_l','jml_dg_p','jml_dg_lp','jml_dgsp_l','jml_dgsp_p','jml_dgsp_lp'], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')

for col in df.columns[~(df.columns.str.contains('%') | (df.columns == 'Kab/Kota'))]:
    df[col] = df[col].str.replace(',', '').astype(int)
    
df

,Kab/Kota,jml_dg_l,jml_dg_p,jml_dg_lp,jml_dgsp_l,jml_dgsp_p,jml_dgsp_lp
33,Kabupaten Bogor,18,68,86,15,25,40
34,Kabupaten Sukabumi,6,11,17,0,0,0
35,Kabupaten Cianjur,0,0,0,0,0,0
36,Kabupaten Bandung,46,122,168,83,134,217


In [50]:
tables = camelot.read_pdf("2018.pdf", flavor='stream', pages='232')

df2 = tables[0].df

df2 = df2.iloc[:26, [1] +  list(range(11, 17))]
df2 = df2.iloc[3:]
df2.set_axis(['Kab/Kota','jml_dg_l','jml_dg_p','jml_dg_lp','jml_dgsp_l','jml_dgsp_p','jml_dgsp_lp'], axis=1, inplace=True)
df2 = df2.replace('', '0')
df2 = df2.replace('-', '0')

for col in df2.columns[~(df2.columns.str.contains('%') | (df2.columns == 'Kab/Kota'))]:
    df2[col] = df2[col].str.replace(',', '').astype(int)

df2
df = pd.concat([df, df2], ignore_index=True)

In [51]:
df_dg = pd.concat([
    df.loc[:,['Kab/Kota','jml_dg_l']].rename(columns={'jml_dg_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi', unit_kerja='rumah sakit'),
    df.loc[:,['Kab/Kota','jml_dg_p']].rename(columns={'jml_dg_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi', unit_kerja='rumah sakit'),
    df.loc[:,['Kab/Kota','jml_dgsp_l']].rename(columns={'jml_dgsp_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi spesialis', unit_kerja='rumah sakit'),
    df.loc[:,['Kab/Kota','jml_dgsp_p']].rename(columns={'jml_dgsp_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi spesialis', unit_kerja='rumah sakit')
], ignore_index=True)

# Menghapus duplikat dan mengurutkan data berdasarkan 'Kab/Kota' dan 'jns_kelamin'
df_dg_rs = df_dg.drop_duplicates().sort_values(['Kab/Kota','jns_kelamin'])

In [52]:
# ## Klinik DIKLAT KESEHATAN
data_bogor = {'Kab/Kota': 'Kabupaten Bogor',
              'jml_dg_l': 0,
              'jml_dg_p': 2,
              'jml_dgsp_l': 0,
              'jml_dgsp_p': 0}

data_sukabumi = {'Kab/Kota': 'Kabupaten Sukabumi',
                'jml_dg_l': 1,
                'jml_dg_p': 3,
                'jml_dgsp_l': 0,
                'jml_dgsp_p': 0}

data_bandung = {'Kab/Kota': 'Kabupaten Bandung Barat',
                'jml_dg_l': 9,
                'jml_dg_p': 20,
                'jml_dgsp_l': 0,
                'jml_dgsp_p': 0}

data_cirebon = {'Kab/Kota': 'Kabupaten Cirebon',
                'jml_dg_l': 1,
                'jml_dg_p': 19,
                'jml_dgsp_l': 0,
                'jml_dgsp_p': 0}

df_bogor = pd.DataFrame([data_bogor])
df_sukabumi = pd.DataFrame([data_sukabumi])
df_cirebon = pd.DataFrame([data_cirebon])
df_bandung = pd.DataFrame([data_bandung])

df_combined = pd.concat([df_bogor, df_sukabumi, df_cirebon, df_bandung], ignore_index=True)

df_dg = pd.concat([
    df_combined.loc[:,['Kab/Kota','jml_dg_l']].rename(columns={'jml_dg_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi', unit_kerja='klinik DINKES'),
    df_combined.loc[:,['Kab/Kota','jml_dg_p']].rename(columns={'jml_dg_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi', unit_kerja='klinik DINKES'),
    df_combined.loc[:,['Kab/Kota','jml_dgsp_l']].rename(columns={'jml_dgsp_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi spesialis', unit_kerja='klinik DINKES'),
    df_combined.loc[:,['Kab/Kota','jml_dgsp_p']].rename(columns={'jml_dgsp_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi spesialis', unit_kerja='klinik DINKES')
], ignore_index=True)

# Menghapus duplikat dan mengurutkan data berdasarkan 'Kab/Kota' dan 'jns_kelamin'
df_dg_dikkes = df_dg.drop_duplicates().sort_values(['Kab/Kota','jns_kelamin'])

In [53]:
df_dg = pd.concat([df_dg_pks,df_dg_rs,df_dg_dikkes], ignore_index=True)
tahun = "2018"
df_dg['tahun'] = tahun
df_dg.to_csv(f'dataset/dg_unit_kerja/{tahun}.csv', index=False)
df_dg

,Kab/Kota,jumlah,jns_kelamin,profesi,unit_kerja,tahun
0,Kabupaten Bandung,12,laki-laki,dokter gigi,puskesmas,2018
1,Kabupaten Bandung,0,laki-laki,dokter gigi spesialis,puskesmas,2018
2,Kabupaten Bandung,56,perempuan,dokter gigi,puskesmas,2018
3,Kabupaten Bandung,0,perempuan,dokter gigi spesialis,puskesmas,2018
4,Kabupaten Bandung Barat,8,laki-laki,dokter gigi,puskesmas,2018
...,...,...,...,...,...,...
227,Kabupaten Cirebon,0,perempuan,dokter gigi spesialis,klinik DINKES,2018
228,Kabupaten Sukabumi,1,laki-laki,dokter gigi,klinik DINKES,2018
229,Kabupaten Sukabumi,0,laki-laki,dokter gigi spesialis,klinik DINKES,2018
230,Kabupaten Sukabumi,3,perempuan,dokter gigi,klinik DINKES,2018


# Data Tahap 1

In [5]:
def check_for_jabatan(page):
    '''
    Fungsi untuk mengecek keberadaan informasi jurusan pilihan
    Input: page (ex: pdf.pages[0])
    Output: 
        - dicitionary berisi informasi jurusan pilihan
    '''
    if "Jumlah Tenaga Kesehatan di Puskesmas" in page.extract_text():
        text = pg.extract_text()
        pendidikan = pg.extract_tables()[1][0][1]
        jurusan_strings = text.split("Jurusan : ")[1].split("Lokasi")[0]
        lokasi_front = ""
        jurusan = jurusan_strings.split("\n")[0]

        lokasi_string = text.split("Lokasi Formasi : ")[1].split("Jenis")[0]

        return {
            "kode_jurusan": jurusan.split(" - ")[0],
            "jurusan": jurusan.split(" - ")[1],
            "kode_lokasi": lokasi_string.split(" - ")[0],
            "lokasi_formasi": lokasi_string.split(" - ")[1].split("\n")[0],
            "pendidikan": pendidikan.replace("\n", " ")
        }
    else:
        return {}

def check_for_detail_tables(page):
    '''
    Fungsi untuk mengecek keberadaan tabel perorangan
    Input: page (ex: pdf.pages[0])
    Output: 
        - found (binary, iya tidaknya sebuah halaman punya tabel perorangan)
        - df_returned (tabel perorangan jika ada)
    '''
    found = False
    df_returned = pd.DataFrame()
    for table in pg.extract_tables():
        df = pd.DataFrame(table)

        if (df.shape[1] == 9): #jumlah kolom = 9 
            found = True
            df_returned = df
    return found, df_returned


def split_df(df_):
    '''
    Fungsi untuk split tabel perorangan. Kadang ada satu halaman dengan lebih dari satu tabel.
    Input: df_ (dataframe tabel perorangan)
    Output: 
        - list berisi beberapa dataframe untuk tiap individu
    '''
    dfs = []
    header_indexes = list(df_[df_[1]=="No Peserta"].index)
    header_indexes.append(len(df_))
    for i in range(len(header_indexes)-1):
        splitted_df = df_.iloc[header_indexes[i]: header_indexes[i+1], :]
        splitted_df.index = range(len(splitted_df))
        dfs.append(splitted_df)
    return dfs

def get_info_from_table(df_, idx):
    '''
    Fungsi untuk mengekstrak informasi dari tabel perorangan 
    Input: df_ (dataframe tabel perorangan)
    Output: 
        - dicitionary berisi informasi perorangan yang telah diekstrak
    '''
    base_data =  {
        "no_peserta": df_.iloc[idx,1],
        "kode_pendidikan": df_.iloc[idx,2],
        "total_twk": df_.iloc[idx,4],
        "total_tiu": df_.iloc[idx,5],
        "total_tkp": df_.iloc[idx,6],
        "total_skd": df_.iloc[idx,7],
        "keterangan": df_.iloc[idx,8]
    }

    return {**base_data}

In [7]:
start_index = 0
end_index = 929
pdf = pdfplumber.open('datasets/Lampiran B-099.02720.06.2022 Pengumuman Hasil Tahap 1 Tahun 2022.pdf')

# start iterating
result = []
last_jabatan = {}

start_time = dt.datetime.now()
start_time = dt.datetime.now()
for i in range(start_index, end_index):
    pg = pdf.pages[i]

    # jika halaman tsb ada info tentang lowongan, simpan
    current_jabatan = check_for_jabatan(pg)
    if current_jabatan != {}:
        last_jabatan = current_jabatan
    
    is_detail_found, detail_df = check_for_detail_tables(pg)
    # jika ketemu ada tabel perorangan
    if is_detail_found:
        splitted_df = split_df(detail_df)
        for df_ in splitted_df:
            for i in range(2, len(df_)):
                details = get_info_from_table(df_, i)
                if current_jabatan == {}:
                    # kalo ada info lowongan di halaman yang sama, pakai info lowongan tsb
                    details.update(last_jabatan)
                else:
                    # kalo ga, pake info lowongan terakhir
                    details.update(current_jabatan)
                    last_jabatan = current_jabatan
                result.append(details)

    # untuk logging
    if i%100 == 99:
        curr_time = dt.datetime.now()
        print("Done for "+str(i), curr_time-start_time)
        start_time = dt.datetime.now()

res = pd.DataFrame(result)


OSError: [Errno 22] Invalid argument: 'spmb_0_929_2024-02-18 13:35:49.467388.csv'

In [9]:
res.to_csv('Tahap 1.csv')